In [2]:
# Cell 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
import xgboost as xgb
import optuna


In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load datasets with raw string paths
train = pd.read_csv(r'C:\Users\yhado\associationsltd.co.uk\OneDrive - associationsltd.co.uk\ML-AI\Final project\event-recommendation-engine-challenge\train.csv')
test = pd.read_csv(r'C:\Users\yhado\associationsltd.co.uk\OneDrive - associationsltd.co.uk\ML-AI\Final project\event-recommendation-engine-challenge\test.csv')
users = pd.read_csv(r'C:\Users\yhado\associationsltd.co.uk\OneDrive - associationsltd.co.uk\ML-AI\Final project\event-recommendation-engine-challenge\users.csv')
user_friends = pd.read_csv(r'C:\Users\yhado\associationsltd.co.uk\OneDrive - associationsltd.co.uk\ML-AI\Final project\event-recommendation-engine-challenge\user_friends.csv.gz', compression='gzip')
events = pd.read_csv(r'C:\Users\yhado\associationsltd.co.uk\OneDrive - associationsltd.co.uk\ML-AI\Final project\event-recommendation-engine-challenge\events.csv.gz', compression='gzip')
event_attendees = pd.read_csv(r'C:\Users\yhado\associationsltd.co.uk\OneDrive - associationsltd.co.uk\ML-AI\Final project\event-recommendation-engine-challenge\event_attendees.csv.gz', compression='gzip')


In [4]:
# Verify that data is loaded correctly
print("Train dataset:")
print(train.head())

print("Test dataset:")
print(test.head())

print("Users dataset:")
print(users.head())

print("User Friends dataset:")
print(user_friends.head())

print("Events dataset:")
print(events.head())

print("Event Attendees dataset:")
print(event_attendees.head())


Train dataset:
      user       event  invited                         timestamp  interested  \
0  3044012  1918771225        0  2012-10-02 15:53:05.754000+00:00           0   
1  3044012  1502284248        0  2012-10-02 15:53:05.754000+00:00           0   
2  3044012  2529072432        0  2012-10-02 15:53:05.754000+00:00           1   
3  3044012  3072478280        0  2012-10-02 15:53:05.754000+00:00           0   
4  3044012  1390707377        0  2012-10-02 15:53:05.754000+00:00           0   

   not_interested  
0               0  
1               0  
2               0  
3               0  
4               0  
Test dataset:
      user       event  invited                         timestamp
0  1776192  2877501688        0  2012-11-30 11:39:01.230000+00:00
1  1776192  3025444328        0  2012-11-30 11:39:01.230000+00:00
2  1776192  4078218285        0  2012-11-30 11:39:01.230000+00:00
3  1776192  1024025121        0  2012-11-30 11:39:01.230000+00:00
4  1776192  2972428928        0  2

In [13]:
# Preprocess the data
# Merge train with events to get start_time from the events dataset
train_merged = pd.merge(train, events[['event_id', 'start_time']], left_on='event', right_on='event_id', how='left')

# Merge the result with users data to include user demographic information
train_merged = pd.merge(train_merged, users, left_on='user', right_on='user_id', how='left')

# Handle missing datetime values in 'start_time' by filling with NaT (Not a Time)
train_merged['start_time'] = pd.to_datetime(train_merged['start_time'], errors='coerce')
train_merged['start_time'] = train_merged['start_time'].fillna(pd.NaT)

# Separate datetime columns from non-datetime columns
# Fill NaN for datetime columns with NaT and for other columns with 0
for col in train_merged.columns:
    if pd.api.types.is_datetime64_any_dtype(train_merged[col]):
        train_merged[col] = train_merged[col].fillna(pd.NaT)
    else:
        train_merged[col] = train_merged[col].fillna(0)

# Drop unnecessary columns, if needed
train_merged.drop(columns=['event_id', 'joinedAt', 'locale'], inplace=True)

# Preview the merged and cleaned dataset
print(train_merged.head())


      user       event  invited                        timestamp  interested  \
0  3044012  1918771225        0 2012-10-02 15:53:05.754000+00:00           0   
1  3044012  1502284248        0 2012-10-02 15:53:05.754000+00:00           0   
2  3044012  2529072432        0 2012-10-02 15:53:05.754000+00:00           1   
3  3044012  3072478280        0 2012-10-02 15:53:05.754000+00:00           0   
4  3044012  1390707377        0 2012-10-02 15:53:05.754000+00:00           0   

   not_interested                       start_time  user_id birthyear gender  \
0               0 2012-10-03 08:00:00.002000+00:00  3044012      1990   male   
1               0 2012-10-03 11:00:00.003000+00:00  3044012      1990   male   
2               0 2012-10-26 13:30:00.003000+00:00  3044012      1990   male   
3               0 2012-10-06 05:00:00.003000+00:00  3044012      1990   male   
4               0 2012-10-06 03:00:00.003000+00:00  3044012      1990   male   

  location  timezone  
0   Binjai     

In [15]:
# Cell 4: Feature engineering
# Create features for the event interactions (attendees, friends, etc.)

# First, fill missing values (NaN) with an empty string
event_attendees['yes'] = event_attendees['yes'].fillna('')
event_attendees['no'] = event_attendees['no'].fillna('')
event_attendees['maybe'] = event_attendees['maybe'].fillna('')

# Then, create new columns with the count of attendees
event_attendees['yes_count'] = event_attendees['yes'].str.split().apply(len)
event_attendees['no_count'] = event_attendees['no'].str.split().apply(len)
event_attendees['maybe_count'] = event_attendees['maybe'].str.split().apply(len)

# Preview the resulting dataframe
print(event_attendees[['yes_count', 'no_count', 'maybe_count']].head())


   yes_count  no_count  maybe_count
0          7         2            7
1         11         0            8
2          0         2            2
3          0         0            0
4          6         1            6


In [27]:
# Cell 5: Prepare training and test sets

# Check available columns in train_merged
print("Columns in train_merged:", train_merged.columns)

# If 'birthyear' is present, convert it to numeric and fill missing values
if 'birthyear' in train_merged.columns:
    train_merged['birthyear'] = pd.to_numeric(train_merged['birthyear'], errors='coerce')
    # Use assignment instead of inplace to avoid the warning
    train_merged['birthyear'] = train_merged['birthyear'].fillna(train_merged['birthyear'].mean())

# Check if 'gender' and 'location' exist before one-hot encoding
categorical_columns = []
if 'gender' in train_merged.columns:
    categorical_columns.append('gender')
if 'location' in train_merged.columns:
    categorical_columns.append('location')

# One-hot encode only if the columns exist
if categorical_columns:
    train_merged = pd.get_dummies(train_merged, columns=categorical_columns, drop_first=True)

# Prepare feature matrix X and target y
X = train_merged.drop(columns=['interested', 'not_interested', 'user', 'event'])
y = train_merged['interested']

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Preview the training data
print(X_train.head())


Columns in train_merged: Index(['user', 'event', 'invited', 'interested', 'not_interested', 'user_id',
       'birthyear', 'timezone', 'timestamp_year', 'timestamp_month',
       ...
       'location_Westminster  California', 'location_Whittier  California',
       'location_Wonogiri  Jawa Tengah  Indonesia',
       'location_Wonosobo  Jawa Tengah  Indonesia',
       'location_Woodland Hills  California', 'location_Yogyakarta',
       'location_Yogyakarta  10', 'location_Yonkers  New York',
       'location_Zhonghe  Heilongjiang  China',
       'location_undefined  undefined'],
      dtype='object', length=408)
       invited     user_id  birthyear  timezone  timestamp_year  \
424          0    97374877       1991    -480.0          2012.0   
554          0   147852768       1997     480.0          2012.0   
2992         0   810608905       1992    -300.0          2012.0   
1954         0   516110926       1987     -60.0          2012.0   
13884        0  3925613292       1993     420.

In [37]:
# Train the XGBoost model with the best hyperparameters from Optuna
import xgboost as xgb
import optuna

# Assuming you have the processed X_train, y_train from earlier
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Optuna optimization
def objective(trial):
    param = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'booster': 'gbtree',
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5)
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    return average_precision_score(y_val, y_pred)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Train model with best parameters
model = xgb.XGBClassifier(**study.best_params)
model.fit(X_train, y_train)


[I 2024-09-12 11:41:54,782] A new study created in memory with name: no-name-6999b5b3-2b82-4321-96f6-aa00e20b1408
[I 2024-09-12 11:41:56,697] Trial 0 finished with value: 0.40538528178906114 and parameters: {'max_depth': 8, 'learning_rate': 0.0011907474514665963, 'n_estimators': 313, 'subsample': 0.5005827333260404, 'colsample_bytree': 0.9427578487569652, 'gamma': 3.0010057488108135}. Best is trial 0 with value: 0.40538528178906114.
[I 2024-09-12 11:41:58,216] Trial 1 finished with value: 0.4176090390151886 and parameters: {'max_depth': 10, 'learning_rate': 0.0038933061491696536, 'n_estimators': 220, 'subsample': 0.5289643168642927, 'colsample_bytree': 0.655233532439276, 'gamma': 3.279097987930151}. Best is trial 1 with value: 0.4176090390151886.
[I 2024-09-12 11:41:59,280] Trial 2 finished with value: 0.37786188924549824 and parameters: {'max_depth': 6, 'learning_rate': 0.0010062976521747345, 'n_estimators': 159, 'subsample': 0.9811536562161998, 'colsample_bytree': 0.9224363189120395,

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9907876923753399, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.4392692124022329,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.07338688961583732,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=239, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [39]:
# Preprocess X_test before making predictions

# Convert 'timestamp' and 'joinedAt' to datetime, then extract useful features
if 'timestamp' in test_merged.columns:
    test_merged['timestamp'] = pd.to_datetime(test_merged['timestamp'], errors='coerce')
    test_merged['timestamp_year'] = test_merged['timestamp'].dt.year
    test_merged['timestamp_month'] = test_merged['timestamp'].dt.month
    test_merged['timestamp_day'] = test_merged['timestamp'].dt.day

if 'joinedAt' in test_merged.columns:
    test_merged['joinedAt'] = pd.to_datetime(test_merged['joinedAt'], errors='coerce')
    test_merged['joinedAt_year'] = test_merged['joinedAt'].dt.year
    test_merged['joinedAt_month'] = test_merged['joinedAt'].dt.month
    test_merged['joinedAt_day'] = test_merged['joinedAt'].dt.day

# Drop the original datetime columns since we now have numeric features
test_merged = test_merged.drop(columns=['timestamp', 'joinedAt'])

# Convert 'birthyear' to numeric and fill missing values
if 'birthyear' in test_merged.columns:
    test_merged['birthyear'] = pd.to_numeric(test_merged['birthyear'], errors='coerce')
    test_merged['birthyear'].fillna(test_merged['birthyear'].mean(), inplace=True)

# One-hot encode categorical columns like 'locale', 'gender', and 'location'
categorical_columns = ['locale', 'gender', 'location']
test_merged = pd.get_dummies(test_merged, columns=categorical_columns, drop_first=True)

# Prepare X_test for prediction
X_test = test_merged.drop(columns=['user', 'event'])  # Make sure you're dropping non-feature columns like 'user' and 'event'

# Now you can make predictions
y_test_pred = model.predict_proba(X_test)[:, 1]

# Output predictions (ranking of events by likelihood of user interest)
test_merged['predicted_interest'] = y_test_pred
recommendations = test_merged[['user', 'event', 'predicted_interest']].sort_values(by=['user', 'predicted_interest'], ascending=[True, False])

# Show the top recommendations
print(recommendations.head(10))


C:\Users\yhado\AppData\Local\Temp\ipykernel_3440\3851784744.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_merged['birthyear'].fillna(test_merged['birthyear'].mean(), inplace=True)


ValueError: feature_names mismatch: ['invited', 'user_id', 'birthyear', 'timezone', 'timestamp_year', 'timestamp_month', 'timestamp_day', 'start_time_year', 'start_time_month', 'start_time_day', 'gender_female', 'gender_male', 'location_  ', 'location_Aberdeen', 'location_Abuja  31', 'location_American Canyon  California', 'location_Anaheim  CA', 'location_Anaheim  California', 'location_Argy  Flacq  Mauritius', 'location_Atlanta  Georgia', 'location_Bajos De Haina  33', 'location_Bajos De Haina  San Cristobal  Dominican Republic', 'location_Balige  Sumatera Utara  Indonesia', 'location_Banda Aceh  Indonesia', 'location_Bandar Lampung', 'location_Bandung  30', 'location_Bandung  Indonesia', 'location_Bandung Dua  Jawa Barat  Indonesia', 'location_Bangalore  India', 'location_Bangko  Jambi  Indonesia', 'location_Bantoel  Yogyakarta  Indonesia', 'location_Bantul', 'location_Banyuwangi  Jawa Timur  Indonesia', 'location_Barboursville  WV', 'location_Barcelona  Spain', 'location_Barrie  On

In [44]:
# Convert birthyear to numeric
test_merged['birthyear'] = pd.to_numeric(test_merged['birthyear'], errors='coerce')

# One-hot encode categorical columns (if not already done)
test_merged = pd.get_dummies(test_merged, columns=['locale', 'gender', 'location'], drop_first=True)

# Ensure the columns match between X_train and X_test
X_test = test_merged.reindex(columns=X_train.columns, fill_value=0)

# Now you can make predictions
y_test_pred = model.predict_proba(X_test)[:, 1]

# Output predictions (ranking of events by likelihood of user interest)
test_merged['predicted_interest'] = y_test_pred
recommendations = test_merged[['user', 'event', 'predicted_interest']].sort_values(by=['user', 'predicted_interest'], ascending=[True, False])

# Display top recommendations
print(recommendations.head(10))



      user       event  predicted_interest
0  1776192  2877501688            0.220296
1  1776192  3025444328            0.220296
2  1776192  4078218285            0.220296
3  1776192  1024025121            0.220296
4  1776192  2972428928            0.220296
5  1776192  2514143386            0.220296
6  1776192  1823369186            0.220296
7  5161061  2027962693            0.227554
8  5161061  1652007005            0.227554
9  5161061  2169802745            0.227554


In [45]:
# Cell for making predictions on the test set
y_test_pred = model.predict_proba(X_test)[:, 1]

# Output predictions (ranking of events by likelihood of user interest)
test_merged['predicted_interest'] = y_test_pred
recommendations = test_merged[['user', 'event', 'predicted_interest']].sort_values(by=['user', 'predicted_interest'], ascending=[True, False])

# Show the top recommendations
print(recommendations.head(10))



      user       event  predicted_interest
0  1776192  2877501688            0.220296
1  1776192  3025444328            0.220296
2  1776192  4078218285            0.220296
3  1776192  1024025121            0.220296
4  1776192  2972428928            0.220296
5  1776192  2514143386            0.220296
6  1776192  1823369186            0.220296
7  5161061  2027962693            0.227554
8  5161061  1652007005            0.227554
9  5161061  2169802745            0.227554


In [46]:
# Cell 7: Train the final model with the best hyperparameters
best_params = study.best_trial.params
final_model = xgb.XGBClassifier(**best_params)
final_model.fit(X, y)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9907876923753399, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.4392692124022329,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.07338688961583732,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=239, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [48]:
# Convert 'birthyear' to numeric
test_merged['birthyear'] = pd.to_numeric(test_merged['birthyear'], errors='coerce')

# Convert 'timestamp' and 'joinedAt' into year, month, and day features
test_merged['timestamp_year'] = pd.to_datetime(test_merged['timestamp'], errors='coerce').dt.year
test_merged['timestamp_month'] = pd.to_datetime(test_merged['timestamp'], errors='coerce').dt.month
test_merged['timestamp_day'] = pd.to_datetime(test_merged['timestamp'], errors='coerce').dt.day

test_merged['joinedAt_year'] = pd.to_datetime(test_merged['joinedAt'], errors='coerce').dt.year
test_merged['joinedAt_month'] = pd.to_datetime(test_merged['joinedAt'], errors='coerce').dt.month
test_merged['joinedAt_day'] = pd.to_datetime(test_merged['joinedAt'], errors='coerce').dt.day

# Drop the original 'timestamp' and 'joinedAt' columns since we've extracted the necessary features
test_merged = test_merged.drop(columns=['timestamp', 'joinedAt'])

# One-hot encode categorical columns like 'locale', 'gender', 'location'
test_merged = pd.get_dummies(test_merged, columns=['locale', 'gender', 'location'], drop_first=True)

# Reindex to ensure X_test matches the columns of X_train
X_test = test_merged.reindex(columns=X_train.columns, fill_value=0)

# Now you can make predictions
y_test_pred = model.predict_proba(X_test)[:, 1]

# Output predictions (ranking of events by likelihood of user interest)
test_merged['predicted_interest'] = y_test_pred
recommendations = test_merged[['user', 'event', 'predicted_interest']].sort_values(by=['user', 'predicted_interest'], ascending=[True, False])

# Display top recommendations
print(recommendations.head(10))




      user       event  predicted_interest
0  1776192  2877501688            0.210123
1  1776192  3025444328            0.210123
2  1776192  4078218285            0.210123
3  1776192  1024025121            0.210123
4  1776192  2972428928            0.210123
5  1776192  2514143386            0.210123
6  1776192  1823369186            0.210123
7  5161061  2027962693            0.359215
8  5161061  1652007005            0.359215
9  5161061  2169802745            0.359215


In [49]:
# Cell 9: Compute Mean Average Precision at 200 for validation set
def mean_average_precision(y_true, y_pred):
    return average_precision_score(y_true, y_pred)

y_pred_val = final_model.predict_proba(X_val)[:, 1]
map_score = mean_average_precision(y_val, y_pred_val)
print(f"Mean Average Precision at 200: {map_score}")


Mean Average Precision at 200: 0.6014549699206235
